## Preprocessing

In [109]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [110]:
# Look at the value_counts of each column. Measure all values greater than 10 counts to create buckets. Investiage any less than 3 to see if they are represented significantly in the data.
bucket_list = []
significance_list = []

for i in application_df.columns:
  if len(application_df.groupby(i)[i].count()) > 10:
    bucket_list.append(i)
  elif len(application_df.groupby(i)[i].count()) < 3:
    significance_list.append(i)



In [111]:
print(bucket_list)
print(significance_list)

['EIN', 'NAME', 'APPLICATION_TYPE', 'CLASSIFICATION', 'ASK_AMT']
['STATUS', 'SPECIAL_CONSIDERATIONS', 'IS_SUCCESSFUL']


In [112]:
# Looking at the data it appears that the SPECIAL_CONSIDERATIONS column isn't significant.
application_df.groupby('SPECIAL_CONSIDERATIONS')['IS_SUCCESSFUL'].count()

SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: IS_SUCCESSFUL, dtype: int64

In [113]:
# Looking at the data it appears that the STATUS column isn't significant.
application_df.groupby('STATUS')['IS_SUCCESSFUL'].count()

STATUS
0        5
1    34294
Name: IS_SUCCESSFUL, dtype: int64

In [114]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', 'SPECIAL_CONSIDERATIONS' AND 'STATUS'.
application_df.drop(columns=['EIN', 'NAME', 'SPECIAL_CONSIDERATIONS', 'STATUS'], inplace = True)

In [115]:
# Create a cutoff function to apply to future cutoffs

def CutOff(cut_off_value, dataframe_column, dictionary_name):

    # Choose a cutoff value and create a list of application types to be replaced
    # use the variable name `application_types_to_replace`
    cutoff_value = cut_off_value
    dictionary_name = []

    # Get the value counts of each application type
    column_counts = dataframe_column.value_counts()

    # Create a list of application types to be replaced
    for type, count in column_counts.items():
        if count < cutoff_value:
            dictionary_name.append(type)

    # Replace in dataframe
    for each in dictionary_name:
        dataframe_column = dataframe_column.replace(each, "Other")

    # Check to make sure binning was successful
    return dataframe_column.value_counts()

In [116]:
print(application_df['ASK_AMT'].value_counts()[:15])

CutOff(25000, application_df['ASK_AMT'], 'amt_to_replace')

5000     25398
10478        3
15583        3
63981        3
6725         3
7003         2
42072        2
5731         2
46327        2
39122        2
11839        2
6742         2
21264        2
86039        2
73987        2
Name: ASK_AMT, dtype: int64


5000     25398
Other     8901
Name: ASK_AMT, dtype: int64

In [117]:
print(application_df['APPLICATION_TYPE'].value_counts()[:15])

CutOff(530, application_df['APPLICATION_TYPE'], 'application_to_replace')

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
Name: APPLICATION_TYPE, dtype: int64


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [118]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
print(application_df['CLASSIFICATION'].value_counts()[:15])

CutOff(290, application_df['CLASSIFICATION'], 'classifications_to_replace')

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
Name: CLASSIFICATION, dtype: int64


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [119]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [120]:
# Split our preprocessed data into our features and target arrays
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
y = application_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [121]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [127]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Number of input features is equal to the shape of a single element of X_train_scaled
input_features = X_train_scaled.shape[1]

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=50, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=24, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_38 (Dense)            (None, 50)                5700      
                                                                 
 dense_39 (Dense)            (None, 24)                1224      
                                                                 
 dense_40 (Dense)            (None, 16)                400       
                                                                 
 dense_41 (Dense)            (None, 16)                272       
                                                                 
 dense_42 (Dense)            (None, 1)                 17        
                                                                 
Total params: 7,613
Trainable params: 7,613
Non-trainable params: 0
_________________________________________________________________


In [128]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [129]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5728 - accuracy: 0.7152
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5517 - accuracy: 0.7302
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5476 - accuracy: 0.7309
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accuracy: 0.7317
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5453 - accuracy: 0.7332
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5438 - accuracy: 0.7343
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5433 - accuracy: 0.7346
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5420 - accuracy: 0.7369
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5415 - accuracy: 0.7334
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5411 - accuracy: 0.7352

In [130]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5528 - accuracy: 0.7315 - 685ms/epoch - 3ms/step
Loss: 0.5527946352958679, Accuracy: 0.7315452098846436


In [131]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityOptimized_4_FINAL.h5")

# .73096